In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

## ------------------ from kaggle:
from subprocess import check_output
print(check_output(["ls", "./"]).decode("utf8"))
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp  # will come in handy due to the size of the data
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


elephant.jpg
first_try.h5
test_train.ipynb
train_example.bson



In [23]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(180,180,3) )

#img_path = 'elephant.jpg'
#img = image.load_img(img_path, target_size=(224, 224))

%matplotlib inline
data = bson.decode_file_iter(open('./train_example.bson', 'rb'))

prod_to_category = dict()
ndim = 180*180*3
counter = 0
feat_arr = []
cat_arr =[]
for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = category_id
    #counter += 1
    for e, pic in enumerate(d['imgs']):
        picture = imread(io.BytesIO(pic['picture']))
        cat_arr.append(product_id)
        #print(e, " ", pic)
        #plt.imshow(picture)
        #plt.show()
        #picture = np.array(picture).reshape(ndim,1)
        #print(picture.shape)
        counter += 1
        # do something with the picture, etc
        x = image.img_to_array(picture)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # decode the results into a list of tuples (class, description, probability)
        # (one such list for each sample in the batch)
        features = model.predict(x)
        feat_arr.append(features)
        #print('Predicted:', decode_predictions(features, top=3)[0])
        #print('Features:', features.shape)
#print(counter)
#print('Predicted:', decode_predictions(features, top=3)[0])


In [25]:
cat_arr = np.asarray(cat_arr)
cat_arr.shape

(110,)

In [30]:
from keras.backend import one_hot
prod_arr = cat_arr
prod_arr_1 = one_hot(prod_arr, 82)

In [31]:
prod_arr_1.shape

TensorShape([Dimension(110), Dimension(82)])

In [33]:
y_dict = dict(zip(list(set(cat_arr)), range(82)))
y_int = np.array([y_dict[category_id] for category_id in cat_arr])

In [36]:
from keras.utils import to_categorical
one_hot_labels = to_categorical(y_int, num_classes=82)

In [37]:
one_hot_labels

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [17]:
model.save_weights('first_try.h5')  # always save your weights after training or during training

In [7]:
feat_arr = np.asarray(feat_arr)
feat_arr.shape
feat_arr_temp = feat_arr

In [13]:
feat_arr_temp = feat_arr_temp.reshape((110,5,5,512))
feat_arr_temp.shape

(110, 5, 5, 512)

In [16]:
model2 = Sequential()
model2.add(Flatten(input_shape=(5,5,512)))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(82, activation='softmax'))
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [41]:
myfirstpred = model2.fit(feat_arr_temp,one_hot_labels)
decode_predictions(myfirstpred)[0]

Epoch 1/10
110/110 [==============================] - 0s - loss: 0.3587 - acc: 0.9772     
Epoch 2/10
110/110 [==============================] - 0s - loss: 0.3569 - acc: 0.9772     
Epoch 3/10
110/110 [==============================] - 0s - loss: 0.3500 - acc: 0.9776     
Epoch 4/10
110/110 [==============================] - 0s - loss: 0.3593 - acc: 0.9774     
Epoch 5/10
110/110 [==============================] - 0s - loss: 0.3530 - acc: 0.9774     
Epoch 6/10
110/110 [==============================] - 0s - loss: 0.3536 - acc: 0.9774     
Epoch 7/10
110/110 [==============================] - 0s - loss: 0.3561 - acc: 0.9774     
Epoch 8/10
110/110 [==============================] - 0s - loss: 0.3578 - acc: 0.9769     
Epoch 9/10
110/110 [==============================] - 0s - loss: 0.3554 - acc: 0.9776     
Epoch 10/10
110/110 [==============================] - 0s - loss: 0.3586 - acc: 0.9774     


NameError: name 'decode_predictions' is not defined

In [42]:
print(category_id)
model.output_shape

101


(None, 5, 5, 512)

In [30]:
feat_arr[0].shape[1:]

(5, 5, 512)

In [19]:

batch_size = 16

generator = datagen.flow_from_directory(
        './',
        target_size=(180, 180),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, 2000)
# save the output as a Numpy array
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

NameError: name 'datagen' is not defined

In [2]:
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)
print(picture.shape)
prod_to_category


from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Using TensorFlow backend.


('Predicted:', [(u'n03584254', u'iPod', 0.28128624), (u'n02992529', u'cellular_telephone', 0.22903134), (u'n02978881', u'cassette', 0.050575905)])


In [3]:
print('Predicted:', decode_predictions(preds, top=10)[0])

('Predicted:', [(u'n03584254', u'iPod', 0.28128734), (u'n02992529', u'cellular_telephone', 0.22903137), (u'n02978881', u'cassette', 0.05057567), (u'n03871628', u'packet', 0.040722415), (u'n04074963', u'remote_control', 0.032242771), (u'n03485407', u'hand-held_computer', 0.024945108), (u'n04548362', u'wallet', 0.023825444), (u'n04404412', u'television', 0.022865141), (u'n04476259', u'tray', 0.017864024), (u'n03666591', u'lighter', 0.013523093)])
